# Assignment 2: Language Modelling in Hangman

Student Name: Hao Tian

Student ID: 1342982

## General info

<b>Due date</b>: Wednesday, 9 April 2025 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python on Google Colab" page on Canvas (under Modules>Resources) for information on how to use Google Colab to use Jupyter Notebook. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages, <b> except </b> Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. You must export your Colab notebook and submit it to the LMS. If your notebook doesn't run, you will lose marks. <b> You should use the Python version that's on Google Colab (3.11)</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given.

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. You <b> must not </b> use artifial intelligence tools for doing the assignment, as all work submitted by a student should be their own, as clarified <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*).

Here's a simple version of the game. **No implementation is needed.**

In [18]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses.

        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))

    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1

        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes

    if verbose:
        print('Out of guesses. The word was', secret_word)
    return mistakes

Here is a human guesser allowing interactive play.

In [19]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [20]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [21]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [22]:
import nltk
from nltk.corpus import brown
import numpy as np

nltk.download('brown')
np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###

# Step 1 :get all words from the brown corpus
all_words = brown.words()

# Step 2 : lowercase all words & remove non-alphabetic characters
all_words = [word.lower() for word in all_words if word.isalpha()]

# Step 3 : get word types
words_types = list(set(all_words))

# Step 4 : randomly shuffle the words
np.random.shuffle(words_types)

# Step 5 : split the words into training and test sets
test_set = words_types[:1000]
training_set = words_types[1000:]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [23]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [24]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (0.5 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you.

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [25]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set.
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [26]:
import string
def random_guesser(mask, guessed, **kwargs):

    ###
    # Your answer BEGINS HERE
    ###
    letter_26_list = list(string.ascii_lowercase)
    rest_characters = []

    for character in letter_26_list:
        if character not in guessed:
            rest_characters.append(character)
    return np.random.choice(rest_characters)
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = mcintosh
Number of mistakes made by the random guesser = 13

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.781


<b>For your testing:</b>

In [27]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters.

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method (use the second answer space). Note that it takes `unigram_counts` as an additional argument.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [28]:
unigram_counts = None
from collections import Counter
###
# Your answer BEGINS HERE
###

"""
In unigram model, we can skip computing actual probabilities
to avoid more computational cost because dividing by the total count
does not change the letters ranking.
"""

# Step 1 : get the frequency of each character in the training set
unigram_counts = Counter()
for word in training_set:
    unigram_counts += Counter(word)
###
# Your answer ENDS HERE
###
def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    # Step 2 : sort the characters by their frequency
    sorted_characters = unigram_counts.most_common()
    for character, frequency in sorted_characters:
        if character not in guessed:
            return character
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.348


<b>For your testing:</b>

In [29]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [30]:
unigram_counts_by_length = None
from collections import defaultdict, Counter

###
# Your answer BEGINS HERE
###

# get a dictionary of character frequency based on word length
unigram_counts_by_length = defaultdict(Counter)
for word in training_set:
    unigram_counts_by_length[len(word)] += Counter(word)

unigram_counts = Counter()
for word in training_set:
    unigram_counts += Counter(word)
###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    word_length = len(mask)  # get the length of the word

    # check if the word length is in the dictionary
    if word_length in unigram_counts_by_length.keys():
         # sorted letters by frequency
        sorted_counts = unigram_counts_by_length[word_length].most_common()
        for each_character, frequency in sorted_counts:
            if each_character not in guessed:
                return each_character
        # trace if no more new chars for this len & return the most frequent
        return unigram_guesser(mask, guessed, unigram_counts)
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.318


<b>For your testing:</b>

In [31]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [32]:
bigram_counts = None
import math
import string
from collections import defaultdict, Counter

###
# Your answer BEGINS HERE
###

# Step1(a). create a 2D dictionary to store the frequency of each character pair
bigram_counts = defaultdict(Counter)

# (b). add padding only on the beginning of bigram and get counts of each pair
for word in training_set:
    padded_word = '$' + word
    for i in range(1, len(padded_word)):
        pre_char = padded_word[i-1]
        current_character = padded_word[i]
        bigram_counts[pre_char][current_character] += 1

###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts):  # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    mask = '$' + ''.join(mask)
    probability_dictionary = defaultdict(float)

    # 1. create letter list
    character_list = list(string.ascii_lowercase)
    rest_characters = list(set(character_list) - guessed)

    # 2. get mask letter
    for character in rest_characters:
        for i in range(1, len(mask)):
            pre_char = mask[i-1]
            current_character = mask[i]

            # judge whether character has been guessed
            if current_character == '_':
                if pre_char != '_':
                    # get statistics W(i-1) & (Wi) joint Counts and W(i) counts

                    # for those never seen word, 0 probability
                    count_joint = bigram_counts.get(pre_char, 0)[character]

                    # to avoid prob = 0 in bigram
                    count_pre = sum(bigram_counts.get(pre_char, 1).values())
                    probability = float(count_joint) / float(count_pre)

                else:
                    probability = (float(unigram_counts[character]) /
                     float(sum(unigram_counts.values())))

                probability_dictionary[character] += probability

    return max(probability_dictionary, key=probability_dictionary.get)
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)



Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.59


### Question 6 (2.0 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 2.0 mark if x < 6.0
* 1.5 mark if 6.0 <= x < 6.5
* 1.0 mark if 6.5 <= x < 7.0
* 0.5 mark if 7.0 <= x < 8.0
* 0.0 mark if x >= 8.0

Note:
* When testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its _average_ performance, as there will be some variance to its performance depending on the training/test split.
* Your code must run under 2 minutes on Colab (free account) with GPU; program that runs longer than that will be terminated and you will score 0.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [33]:
###
# Your answer BEGINS HERE
###

from collections import defaultdict, Counter
import string
import random
import math

# Step 1: Initialize counts for 1,2,3,4 grams

unigram_counts = Counter()

# (a).Forward 1,2,3,4 grams count
bigram_counts = defaultdict(Counter)
trigram_counts = defaultdict(Counter)
fourgram_counts = defaultdict(Counter)

# (b). Reverse 1,2,3,4 grams count
bigram_r_counts = defaultdict(Counter)
trigram_r_counts = defaultdict(Counter)
fourgram_r_counts = defaultdict(Counter)

# Process each word in the training set to count N-grams
for word in training_set:
    unigram_counts.update(word)  # Update unigram counts

    # Add pedding for forward 1,2,3,4 grams
    padded = "$$$" + word + "$"
    for i in range(1, len(padded)):
        bigram_counts[padded[i - 1]][padded[i]] += 1
    for i in range(2, len(padded)):
        trigram_counts[padded[i - 2:i]][padded[i]] += 1
    for i in range(3, len(padded)):
        fourgram_counts[padded[i - 3:i]][padded[i]] += 1

    # Add pedding for reverse 1,2,3,4 grams
    reversed_word = word[::-1]
    padded_rev = "$$$" + reversed_word + "$"
    for i in range(1, len(padded_rev)):
        bigram_r_counts[padded_rev[i - 1]][padded_rev[i]] += 1
    for i in range(2, len(padded_rev)):
        trigram_r_counts[padded_rev[i - 2:i]][padded_rev[i]] += 1
    for i in range(3, len(padded_rev)):
        fourgram_r_counts[padded_rev[i - 3:i]][padded_rev[i]] += 1

def my_amazing_ai_guesser(mask, guessed):
    padded = "$$$" + ''.join(mask) + "$"

    # Weight balance between forward and reverse models
    alpha = 0.5
    smoothing = 0.15

    # Dictionary to record scores for each candidate character
    scores = defaultdict(float)

    # List of the rest of unguessed characters
    candidates = list(set(string.ascii_lowercase) - guessed)

    # Function to calculate log probability of a character in a given context
    def log_prob(counts, context, c):
        return math.log((counts[context][c] + smoothing)
        / (sum(counts[context].values()) + 26 * smoothing))

    # Calculate scores for each candidate character
    for c in candidates:
        for i in range(3, len(padded) - 1):  # Start from the 4th character
            if padded[i] != '_':  # Skip empty_place characters
                continue

            # Extract context for the character (left and right N-grams)
            left4 = padded[i - 3:i]  # for 4-grams
            left3 = left4[-3:]  # for 3-grams
            left1 = left4[-1]  # for 2-grams

            # Etract Reverse for character
            right4 = padded[i + 1:i + 4][::-1]  # reverse 4-grams
            right3 = right4[-3:]  # reverse 3-grams
            right1 = right4[-1]  # reverse 2-grams

            # Calculate probability using different N-grams based
            # based on current known characters number

            if '_' not in left4 and left4 in fourgram_counts:
                left_prob = log_prob(fourgram_counts, left4, c)
            elif '_' not in left3 and left3 in trigram_counts:
                left_prob = log_prob(trigram_counts, left3, c)
            elif left1 in bigram_counts:
                left_prob = log_prob(bigram_counts, left1, c)
            else:
                left_prob = math.log((unigram_counts[c] + smoothing)
                 / (sum(unigram_counts.values()) + 26 * smoothing))

            if '_' not in right4 and right4 in fourgram_r_counts:
                right_prob = log_prob(fourgram_r_counts, right4, c)
            elif '_' not in right3 and right3 in trigram_r_counts:
                right_prob = log_prob(trigram_r_counts, right3, c)
            elif right1 in bigram_r_counts:
                right_prob = log_prob(bigram_r_counts, right1, c)
            else:
                right_prob = math.log((unigram_counts[c] + smoothing)
                 / (sum(unigram_counts.values()) + 26 * smoothing))

            # the forward and reverse probabilities with same weight
            scores[c] += alpha * left_prob + (1 - alpha) * right_prob

    # Return the character with the highest score or choose randomly if no score
    return max(scores, key=scores.get) if scores else random.choice(candidates)

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)


Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  6.419


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

#### 1. Approach Clarification & Result:

##### (a). My approach uses a bi-directional N-gram model (forward and reverse) for character prediction, based on surrounding context. In the preprocessing step, I pad each word with special characters ($$$) to account for boundaries and then calculate the 1-gram, 2-gram, 3-gram, and 4-gram probabilities in both forward and reverse directions.  These probabilities are then used to estimate the likelihood of each unguessed character.

##### (b). For each prediction, the algorithm preferentially matches the largest N-gram (i.e., starting with 4-gram and demoting to 3-gram, 2-gram, and 1-gram if there is no match) to obtain the most accurate context information. If no larger N-gram is matched, the algorithm uses a smaller N-gram to ensure that the context information is as complete as possible.

#### 2. Hyper-parameter & Result:
##### (a). I applied log-probability to avoid possible underflow problems caused by n-grams and smoothing to avoid zero probabilities for unseen N-grams. And the smoothing parameter (0.15) was tuned to balance between underfitting and overfitting. The combination of forward and reverse probabilities was calculated by the same weight importance, with alpha = 0.5 to balance the contribution of both directions. The character with the highest combined score is selected as the guess.

##### (b). The Average number of incorrect guesses with seed=1 on Colab : 6.419


##### Your answer ENDS HERE